# Jax jit

## Lesson Goals:

By the end of this lesson, you will know how to use the `jit`, how to accurately time computations using `jit`-ted functions, and how to identify where to `jit` things. In the process, we will quickly discuss functional programming and why functional programming is useful for speeding up computations. 

## Core Concepts



## Concepts In action:


- Easy: [lotka-volterra](../case_studies/lotka-volterra/README.md)

- Intermediate: [leaky_integrate_and_fire](../case_studies/leaky_integrate_and_fire/README.md)

- Advanced: [gaussian_mixture_model](../case_studies/gaussian_mixture_model/README.md)

In [3]:
from typing import TypeAlias
import time
import jax.numpy as jnp
import numpy as np
import jax

np.random.seed(42)

# Functional Programming?

Functional programming is many things, but for the purposes of this tutorial, it is a form of programming without side-effects. Python is not a functional programming language, but you may have heard of others such as `haskell`, `ocaml`, or `erlang`.

The most common form of side-effects involves modifying some internal state. Consider the following:

In [4]:
import copy
class ShoppingCart:
    def __init__(self):
        self.items = []

    def add_item(self, item):
        self.items.append(item)  # Side effect: modifying internal state

    def __repr__(self):
        return f"ShoppingCart({self.items})"

cart = ShoppingCart()
cart.add_item("banana")
print(cart)
cart.add_item("apple")
print(cart)
print("*" * 10)

class FunctionalShoppingCart:
    def __init__(self):
        self.items = []

    def add_item(self, item):
        new_cart = FunctionalShoppingCart()
        all_items = copy.deepcopy(self.items)
        all_items.append(item)
        new_cart.items = all_items
        return new_cart

    def __repr__(self):
        return f"FunctionalShoppingCart({self.items})"

func_cart = FunctionalShoppingCart()
func_cart.add_item("banana")  # <- The banana was not added!
print(func_cart)
func_cart = func_cart.add_item("apple")
print(func_cart)

ShoppingCart(['banana'])
ShoppingCart(['banana', 'apple'])
**********
FunctionalShoppingCart([])
FunctionalShoppingCart(['apple'])


## Functional Programming:

Okay, but how is this relevant? Well, functional programming allows for:

- predictable behavior: compilers can more easily optimize your code
- immutability: the data cannot be modified, so all threads/ processes just grab a copy of the original data and process it async.

# Jax's JIT: Supercharged functions

A `jit` is a just-in-time compilation of your code. `Python` is famously slow because, among other things, the code is interpreted i.e. at run-time, the interpreter has to decide what to do. Languages like `C++` are `Rust` are compiled so at run-time, the code is just... run.

So, by compiling out Jax code via the `jit`, we can accelerate our programs. Assuming the numerical computation is the bottleneck, as is often the case in ML tasks, this means that we have sped up the slowest part of our program.

## Where does functional programming come in? 

FP makes it easier for the `jit` compiler to speed up the code. It can do things like:

- function inlining: the function call is replaced by the function itself

- loop fusion/elimination/unrolling: by removing dependencies between calls, jax can 

- memoization: jax can cache results for particular inputs and return those if it sees those particular inputs again

How do we use the jit? We can either use it as a function decorator, or as a function call. Each has its own advantages and disadvantages

In [5]:
def doubler(x):
    return x * 2

@jax.jit
def jitted_doubler(x):
    return x * 2

alternative_jitted_doubler = jax.jit(doubler)

# Jit in action

Let's do a simple-ish task where we generate a matrix, $M \in [0, 1)^{1000, 1000}$ and everything less than 0.5 we take the square-root of, and anything greater than 0.5 we square. We finally multiply this matrix with itself

In [6]:
input_arr = np.random.rand(1000, 1000)

def func_np(m):
    mask = m > 0.5
    m = np.where(mask, m**2, np.sqrt(m))
    return m @ m

print("Numpy version")
%timeit func_np(input_arr)

input_arr_j = jnp.asarray(input_arr)

def func_jax(m):
    # TODO: implement the jax equivalent of the above
    pass

print("Jax Non-Jit version")
%timeit func_jax(input_arr_j)

jitted_func = ... # Your code here

print("Jax Jitted version")
%timeit jitted_func(input_arr_j).block_until_ready()

Numpy version
37.7 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Jax Non-Jit version
29.9 ns ± 0.878 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
Jax Jitted version


TypeError: 'ellipsis' object is not callable

# Jit in action Part 2

The previous task was pretty trivial for a modern computer. Let's increase the size of M. $M \in [0, 1)^{4K, 4K}$, increasing the size by 16X

In [ ]:
input_arr = np.random.rand(4_000, 4_000)
print("Numpy version")
%timeit func_np(input_arr)

input_arr_j = jnp.asarray(input_arr)

print("Jax Non-Jit version")
%timeit func_jax(input_arr_j)

jitted_func = jax.jit(func_jax)

print("Jax Jitted version")
%timeit jitted_func(input_arr_j).block_until_ready()

## Quick Aside: Benchmarking in Jax

The astute would have noticed the `.block_until_ready()` function call. What gives? Well, jax returns a future to prevent blocking the main python thread. So, to get accurate timings we had to use the `.block_until_ready()`. To ensure that you get accurate timings when benchmarking you can:

- use `.block_until_ready()`
- convert the `jnp.array` into `np.array` to wait for the future
- print the `jnp.array`

For more information check out: [Jax Async Dispatch](https://jax.readthedocs.io/en/latest/async_dispatch.html)

# Exercise: Using side-effects

To really drill in the idea of mutability and immutability, we're going to show how important it is to be immutable when you use `jit`. Define your own custom function that relies on some external state, as well as its `jit` version. We'll then see how they diverge.  

In [7]:
small_rand_jnp = jnp.asarray(np.random.rand(3, 3))


In [9]:

def pure(x):
    raise NotImplemented

def impure(x):
    raise NotImplemented


for i in range(5):
    print(pure(small_rand_jnp))
    print(impure(small_rand_jnp))
    print("*" * 10)
    
    # TODO: update the state here and see how they diverge!
    
    

    

[[3.5421020e-01 1.3301860e-01 2.8897290e-05]
 [3.1481943e-01 8.0383843e-01 2.8272283e-01]
 [6.0916096e-01 2.6229199e-02 1.8850651e-02]]
[[0.9493658  0.49773574 0.00540452]
 [0.8759072  1.7004089  0.8144397 ]
 [1.3896487  0.1881835  0.15614831]]
**********
[[0.9493658  0.49773574 0.00540452]
 [0.8759072  1.7004089  0.8144397 ]
 [1.3896487  0.1881835  0.15614831]]
[[0.9493658  0.49773574 0.00540452]
 [0.8759072  1.7004089  0.8144397 ]
 [1.3896487  0.1881835  0.15614831]]
**********
[[1.5445215  0.8624529  0.01078014]
 [1.4369948  2.5969794  1.3461566 ]
 [2.1701362  0.35013783 0.293446  ]]
[[0.9493658  0.49773574 0.00540452]
 [0.8759072  1.7004089  0.8144397 ]
 [1.3896487  0.1881835  0.15614831]]
**********
[[2.139677   1.2271701  0.01615576]
 [1.9980825  3.4935496  1.8778734 ]
 [2.9506238  0.5120922  0.43074366]]
[[0.9493658  0.49773574 0.00540452]
 [0.8759072  1.7004089  0.8144397 ]
 [1.3896487  0.1881835  0.15614831]]
**********
[[2.7348325  1.5918871  0.02153138]
 [2.5591702  4.39012 

# Further Exercises:

0) Read through [case_studies/leaky_integrate_and_fire/jax_leaky_integrate_and_fire_2_jit.ipynb](../case_studies/leaky_integrate_and_fire/jax_leaky_integrate_and_fire_2_jit.ipynb)

1) Read through [extras_when_not_to_jit.ipynb](./extras_when_not_to_jit.ipynb)

2) Read through [Jax AoT Compiling](https://jax.readthedocs.io/en/latest/aot.html) and take note of the limitations that come with Jax's AoT compiling